# Project 4 - Twitter NLP

Name: Robert Fang
Date: May 10, 2019

In [2]:
#!pip install tweepy

In [1]:
import tweepy
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
from IPython.display import clear_output
import pickle

In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [3]:
import twitter_credentials

In [4]:
auth = tweepy.OAuthHandler(twitter_credentials.consumer_key,twitter_credentials.consumer_secret)
auth.set_access_token(twitter_credentials.access_token,twitter_credentials.access_secret)

api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [5]:
query = ('#OOTT -filter:retweets')

In [92]:
tweet_data = []
count=0
want= 100
while len(tweet_data)<want :
    for tweet in tweepy.Cursor(api.search,q=query,include_entities=True, tweet_mode='extended', count=100).items() :
        d = {'screen_name':tweet.user.screen_name,
             'tweet_id':tweet.id,
             'user_name':tweet.user.name,
             'created':tweet.created_at,
             'text':tweet.full_text,
             'coord':tweet.coordinates,
             'user_loc':tweet.user.location,
             'user_id':tweet.user.id_str,
             'rt_count':tweet.retweet_count,
             'rt_by':tweet.retweeted,
             'phone_type':tweet.source,
             'favs':tweet.favorite_count,
             'fav_by':tweet.favorited,
             'replied':tweet.in_reply_to_status_id_str,
             'hashtags':tweet.entities['hashtags'],
             'lang':tweet.lang,
             'urls':tweet.user.entities,
             'about':tweet.user.description,
             'v':tweet.user.verified,
             'friends':tweet.user.friends_count,
             'followers':tweet.user.followers_count,
             'origin':tweet.user.created_at
            }
        tweet_data.append(d)
        count+=1
        clear_output()
        print(count)
        if len(tweet_data) >= want :
            break

100


In [93]:
df_tweets_add = pd.DataFrame(tweet_data)

In [94]:
#df_tweets = pd.concat([df_tweets, df_tweets_new], ignore_index=False, sort=False)

In [95]:
df_tweets_add.shape

(100, 22)

In [107]:
print(df_tweets_add['text'][16])

Oil prices fell on Wednesday after industry data showed an increase in #US crude inventories https://t.co/7MriWN9i73 #OOTT https://t.co/uYp9eZEchw


## Load the tweets into Mongo DB in AWS

In [47]:
from pymongo import MongoClient
from pymongo import InsertOne, DeleteOne, ReplaceOne
from pprint import pprint

In [48]:
config = {
  'host': '13.59.224.65:27017',
  'username': 'robcfang',
  'password': 'mongodb',
  'authSource': 'cool'
}

client = MongoClient(**config)

db = client.cool

##### Create a collection in AWS mongodb cool database called tweets

In [49]:
db.tweets

Collection(Database(MongoClient(host=['13.59.224.65:27017'], document_class=dict, tz_aware=False, connect=True, authsource='cool'), 'cool'), 'tweets')

##### Select Database called OOTT

In [50]:
db_oil = client['OOTT']

In [55]:
#dblist = client.list_database_names()

In [ ]:
# insert them into AWS MongoDB
db.tweets.insert_many(tweet_data)

In [56]:
cursor =  db['tweets'].find({})

In [57]:
df_mongo = pd.DataFrame(list(cursor))

In [58]:
df_mongo.head()

,_id,about,coord,created,fav_by,favs,followers,friends,hashtags,lang,...,rt_by,rt_count,screen_name,text,tweet_id,urls,user_id,user_loc,user_name,v
0,5ce1bc1fbcbaf61b5aa8cf79,"Market data, analysis, and commentary from Sau...",None,2019-05-19 20:15:03,False,1,19685,9,[],en,...,False,0,ArgaamPlus,UAE Energy Minister @HESuhail says the job by ...,1130205245567590402,{'url': {'urls': [{'url': 'http://t.co/hHTSFHi...,2872782998,,Argaam Plus,True
1,5ce1bc1fbcbaf61b5aa8cf7a,,None,2019-05-19 20:03:27,False,0,130,47,"[{'text': 'oil', 'indices': [72, 76]}]",en,...,False,1,SpiroSpianada,https://t.co/WmTTh6Htka\n\nSaudis say there's ...,1130202328781471746,{'description': {'urls': []}},541598729,chicago,Spiro I,False
2,5ce1bc1fbcbaf61b5aa8cf7b,Real-time #oilprice news & analysis on #oilmar...,None,2019-05-19 20:02:49,False,1,114676,678,"[{'text': 'oil', 'indices': [47, 51]}]",en,...,False,6,PlattsOil,Saudis say there's consensus to push for lower...,1130202168533889025,{'url': {'urls': [{'url': 'https://t.co/k4uMv1...,22032260,"New York, USA",Platts Oil,False
3,5ce1bc1fbcbaf61b5aa8cf7c,Connecting you to the pipeline industry worldwide,None,2019-05-19 20:00:26,False,0,23410,347,"[{'text': 'pipeline', 'indices': [69, 78]}]",en,...,False,1,PipelineandGas,Regulators in New York have denied a permit fo...,1130201566777151489,{'url': {'urls': [{'url': 'https://t.co/36HXvu...,82712805,"Houston, TX",Pipeline&Gas Journal,False
4,5ce1bc1fbcbaf61b5aa8cf7d,Opec and MENA oil and gas correspondent for @A...,None,2019-05-19 19:59:54,False,0,4148,1037,[],en,...,False,1,ncitayim,NEW: Russian Urals crude shipments to Poland a...,1130201432131608576,{'description': {'urls': []}},1530077334,"دبي, الامارات العربية المتحدة",Nader Itayim | ‌‌نادر ایتیّم,True


## Another Method Saving it down to local MongoDB

In [59]:
# This connects us to the "OOTT" database, and the "tweets" in that database
client = MongoClient()

##### Select collection called Tweets in the OOTT database

In [60]:
tweet_collection = db_oil.tweets

In [62]:
#db_oil.list_collection_names()

In [64]:
# insert them into Local MongoDB
db_oil.tweets.insert_many(tweet_data)

In [678]:
tweet_collection.count_documents({})

110294

In [679]:
#Check to see how many tweets have OPEC in them
tweet_collection.count_documents({'text': {'$regex': 'OPEC'}})

6315

In [680]:
# aggregate is a pipeline, order matters, count the top 20 hastags
cursor_hash = tweet_collection.aggregate([
    {'$match': {'hashtags.0': {'$exists': 1}}},
    {'$project': {'_id': 0, 'hashtags': '$hashtags'}},
    {'$unwind': '$hashtags'},
    {'$group': {'_id': '$hashtags.text', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 20}
])

list(cursor_hash)

[{'_id': 'OOTT', 'count': 46459},
 {'_id': 'oott', 'count': 8873},
 {'_id': 'oil', 'count': 7673},
 {'_id': 'WTI', 'count': 6539},
 {'_id': 'CL_F', 'count': 5423},
 {'_id': 'OIL', 'count': 4584},
 {'_id': 'OPEC', 'count': 3917},
 {'_id': 'Oil', 'count': 3258},
 {'_id': 'Iran', 'count': 2838},
 {'_id': 'CL', 'count': 2292},
 {'_id': 'China', 'count': 1994},
 {'_id': 'crude', 'count': 1937},
 {'_id': 'CRUDE', 'count': 1871},
 {'_id': 'ES_F', 'count': 1843},
 {'_id': 'oilgas', 'count': 1774},
 {'_id': 'Saudi', 'count': 1515},
 {'_id': 'Brent', 'count': 1497},
 {'_id': 'LNG', 'count': 1449},
 {'_id': 'EIA', 'count': 1347},
 {'_id': 'crudeoil', 'count': 1246}]

In [681]:
cursor =  db_oil['tweets'].find({})

In [685]:
df_mongo = pd.DataFrame(list(cursor))

In [686]:
df_mongo.head()

,_id,about,coord,created,fav_by,favs,followers,friends,hashtags,lang,...,rt_by,rt_count,screen_name,text,tweet_id,urls,user_id,user_loc,user_name,v
0,5cdb18e1dba9421e562f3c7b,Christian. Uruguayan 🇺🇾 🇮🇹 . International Rel...,None,2019-05-14 19:24:52,False,0,47,584,"[{'text': 'Fujairah', 'indices': [73, 82]}]",en,...,False,2,Luca3_16,RT @katiemcque: There’s a rumour going around ...,1128380677072850946,{'description': {'urls': []}},1084257893803872257,República Oriental del Uruguay,Luca Fernández 🇺🇾,False
1,5cdb18e1dba9421e562f3c7c,Bloomberg News Executive Editor for Energy & C...,None,2019-05-14 19:24:35,False,0,15454,881,"[{'text': 'OOTT', 'indices': [92, 97]}]",en,...,False,4,StuartLWallace,RT @liamdenning: The remarkably laid-back oil ...,1128380604972662784,{'url': {'urls': [{'url': 'http://t.co/mEEwYlj...,2531942016,London,Stuart Wallace,True
2,5cdb18e1dba9421e562f3c7d,Ayega toh Modi hi.,None,2019-05-14 19:20:20,False,0,1531,393,[],en,...,False,136,Oye_Jahazi,RT @StuartLWallace: Two Saudi oil tankers were...,1128379535404208128,{'url': {'urls': [{'url': 'https://t.co/PZI75I...,497779475,"Punjab, India",Shivam Sharma,False
3,5cdb18e1dba9421e562f3c7e,MENA energy/OPEC analyst at Medley Global Advi...,None,2019-05-14 19:18:26,False,0,522,1020,"[{'text': 'Fujairah', 'indices': [73, 82]}]",en,...,False,2,darwazah,RT @katiemcque: There’s a rumour going around ...,1128379060265070592,{'description': {'urls': []}},152741746,"New York, NY",Mohammad Darwazah,False
4,5cdb18e1dba9421e562f3c7f,"Medical tattooing for #scalpmicropigmentation,...",None,2019-05-14 19:17:42,False,0,92,98,"[{'text': 'women', 'indices': [80, 86]}, {'tex...",en,...,False,1,ink_illusion,RT @thedanieladoll: Why do some women lose the...,1128378876110024704,{'url': {'urls': [{'url': 'https://t.co/S4Lq6B...,1012752318071177216,London & Nashville,Ink Illusions,False


## CHECKPOINT - Save Tweet Dataframe down as Pickle

In [70]:
#pickle dataframe as a checkpoint
with open('proj4_tweet_2.pickle', 'wb') as to_write:
    pickle.dump(df_tweets_add, to_write)

## StreamListener for Live Tweets

In [81]:
from tweepy import Stream
from tweepy.streaming import StreamListener
from IPython import display
from collections import deque
import json

class MyListener(StreamListener):
    def __init__(self):
        super().__init__()
        self.list_of_tweets = deque([], maxlen=5)
        
    def on_data(self, data):
        tweet_text = json.loads(data)['text']
        self.list_of_tweets.append(tweet_text)
        self.print_list_of_tweets()
        #self.collections.insert_one(tweet_text)
    
    def on_error(self, status):
        print(status)

    def print_list_of_tweets(self):
        display.clear_output(wait=True)
        for index, tweet_text in enumerate(self.list_of_tweets):
            m='{}. {}\n\n'.format(index, tweet_text)
            print(m)
            
twitter_stream = Stream(auth, MyListener(), tweet_mode='extended')
twitter_stream.filter(track=['#OOTT'])

0. Crude #Oil prices have pulled back after the weekly energy inventory reports, which showed supplies growing faster… https://t.co/hrg5SqOwcU


1. Crude #Oil prices have pulled back after the weekly energy inventory reports, which showed supplies growing faster… https://t.co/emIPuUu47n


2. Interesting. I remember $SD discussing in their March, 2014 analyst day saving ~$15,000 by no longer painting their… https://t.co/zdZID1TbDF


3. Crude #Oil prices have pulled back after the weekly energy inventory reports, which showed supplies growing faster… https://t.co/wc0oDldJ7M


4. Crude #Oil prices have pulled back after the weekly energy inventory reports, which showed supplies growing faster… https://t.co/wgTvLKlOGe




KeyboardInterrupt: 